<h1><center> Foursquare Location Matching </center></h1>
<h3><center> Train Data Preprocessing </center></h3>
<h3><center> Tao Shan </center></h3>

This notebook preprocess data pairs including methods:
1. finding same countries pairs
1. generate new cols accroding to missing values
1. numeric data generate new columns
1. impute missing values
1. word preprocessing
1. fuzzy similarity
1. save data to pkl format. This format runs faster than csv

### Other Relevant notebooks and links

Competition: [Foursquare - Location Matching](https://www.kaggle.com/competitions/foursquare-location-matching)

Train data generation notebook: [Foursquare - train data generation](https://www.kaggle.com/taos2000/foursquare-train-data-generation)

Train data preprocessing notebook: [Foursquare - train data preprocess](https://www.kaggle.com/taos2000/foursquare-train-data-preprocess)

Model Selection: [Foursquare - model selection](https://www.kaggle.com/taos2000/foursquare-model-selection)

Model Training: [Foursquare - model training](https://www.kaggle.com/taos2000/foursquare-model-training)

In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from geopy.geocoders import Nominatim

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer

import Levenshtein as lev
import math
from collections import Counter

from pickle import dump, load
import time
from sklearn.neighbors import BallTree


import itertools
from tqdm.auto import tqdm
tqdm.pandas()
import gc

from fuzzywuzzy import fuzz
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
start_time = time.time()

In [4]:
#pairs = pd.read_csv('../input/foursquare-location-matching/pairs.csv')
#pairs = pd.read_csv('../input/foursquare-location-matching/pairs.csv').sample(n=100000, random_state=6888)
pairs = pd.read_pickle('../input/last-day-four-points/train_pairs (1).pkl')

In [5]:
ids = ['id_1','id_2']
test_id = pairs[ids]
pairs['country_same'] = np.where(pairs['country_1'].astype(object) == pairs['country_2'].astype(object),1,0)
# missing values generate new cols
missing_list = ['url_1','url_2','phone_1','phone_2','address_1','address_2','city_1','city_2','zip_1','zip_2']
for col in tqdm(missing_list):
    pairs[f"{col}_missing"] = pairs[col].notnull().astype('int8')
def count_occurance(df, cols_1,cols_2):
    for i in tqdm(range(len(cols_1))):
        df[f"{cols_1[i]}_count"] = (df[cols_1[i]].map(df[cols_1[i]].dropna().value_counts().to_dict())).fillna(0)
        df[f"{cols_2[i]}_count"] = df[cols_2[i]].map(df[cols_2[i]].dropna().value_counts().to_dict()).fillna(0)
        df[f"{cols_1[i]}_count_diff"] = (df[f"{cols_2[i]}_count"] - df[f"{cols_1[i]}_count"])
        gc.collect()
    return df
count_cols_1 = ['country_1','city_1','state_1','categories_1']
count_cols_2 = ['country_2','city_2','state_2','categories_2']
pairs = count_occurance(pairs, count_cols_1, count_cols_2)
def numeric_group_counts(df,cols_1,cols_2):
    # cols should be [lat,lon]
    for i in tqdm(range(len(cols_1))):
        group_1 = pd.cut(df[cols_1[i]], 180)
        df[f"{cols_1[i]}_count"] = group_1.map(group_1.value_counts().to_dict())
        group_2 = pd.cut(df[cols_1[i]], 180)
        df[f"{cols_2[i]}_count"] = group_2.map(group_2.value_counts().to_dict())
        df[f"{cols_2[i]}_count_diff"] = (df[f"{cols_1[i]}_count"] - df[f"{cols_2[i]}_count"])
        gc.collect()
    return df
num_group_count_1 = ['latitude_1','longitude_1']
num_group_count_2 = ['latitude_2','longitude_2']
pairs = numeric_group_counts(pairs, num_group_count_1, num_group_count_2)
gc.collect()
# impute missing values
cat_col = pairs.select_dtypes(include = ['object']).columns
pairs[cat_col] = pairs[cat_col].fillna('')
gc.collect()
# 1. location
def distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a = (np.sin(d_lat/2)) ** 2 + np.cos(lat1) * np.cos(lat2) * (np.sin(d_lon/2)) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

# lowercase
def lower(df, cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: x.lower())
    return df

# number removing
def num_remove(df, cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: re.sub(r'\d+', '', x))
    return df

# punctuation removal
def punc_remove(df,cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: re.sub('['+string.punctuation+']', ' ', x))
    return df

# white spaces removal
def space_remove(df,cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: x.strip()) # remove front and end space
        df[col] = df[col].str.replace('\s+', ' ', regex=True) # remove double space
    return df

def preprocess(df,cols):
    df[cols] = num_remove(df,cols)[cols]
    df[cols] = punc_remove(df,cols)[cols]
    df[cols] = space_remove(df,cols)[cols]
    return df

# remove url
def remove_URL(df,cols):
    # cols = ["url_1","url_2"]
    df[cols] = df[cols].fillna('')
    for i in tqdm(cols):
        df[i] = df[i].str.replace('http', '')
        df[i] = df[i].str.replace('https', '')
        df[i] = df[i].str.replace('www', '')
        #df[i] = df[i].progress_apply(lambda x: re.sub('\W', "", x))
    return df

# stop words removal

def list_to_string(lis):
    string = ''
    for i in tqdm(lis):
        string += i
        string += ' '
    return string[:-1]

def stop(string):
    stops = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    result = [i for i in tqdm(tokens) if not i in stops]
    return result
    
def stop_remove(df,cols):
    stops = set(stopwords.words('english'))
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: ' '.join([word for word in x.split() if word not in stops]))
    return df
lowercase_cols = ['name_1','address_1','city_1','state_1','url_1','categories_1','name_2','address_2','city_2','state_2','url_2','categories_2']
preprocess_cols = ['name_1','address_1','name_2','address_2','url_1','url_2','categories_1','categories_2']
url_columns = ['url_1','url_2']
pairs['distance'] = distance(pairs.latitude_1,pairs.longitude_1,pairs.latitude_2,pairs.longitude_2)
pairs[lowercase_cols] = lower(pairs[lowercase_cols], lowercase_cols)[lowercase_cols]
pairs[preprocess_cols] = preprocess(pairs[preprocess_cols], preprocess_cols)[preprocess_cols]
pairs[url_columns] = remove_URL(pairs[url_columns], url_columns)[url_columns]
pairs[preprocess_cols] = stop_remove(pairs[preprocess_cols], preprocess_cols)[preprocess_cols]
def fuzzy_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    temp = pd.DataFrame()
    for i in tqdm(range(len(cols_1))):
        temp[f"{cols_1[i]}_fuzzy"] = df.progress_apply(lambda x: lev.ratio(x[cols_1[i]],x[cols_2[i]]), axis = 1)
        gc.collect()
    return temp    
def fuzzy_similarity_partial(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    temp = pd.DataFrame()
    for i in tqdm(range(len(cols_1))):
        temp[f"{cols_1[i]}_fuzzy_partial"] = df.progress_apply(lambda x: fuzz.partial_ratio(x[cols_1[i]],x[cols_2[i]]), axis = 1)
        gc.collect()
    return temp    

col_1 = ['name_1','address_1','categories_1','url_1']
col_2 = ['name_2','address_2','categories_2','url_2']
col_1_partial = ['name_1','categories_1']
col_2_partial = ['name_2','categories_2']
temp = fuzzy_similarity(pairs[col_1+col_2], col_1, col_2)
pairs = pd.concat([pairs,temp], axis = 1)
del temp
gc.collect()
temp = fuzzy_similarity_partial(pairs[col_1_partial+col_2_partial], col_1_partial, col_2_partial)
pairs = pd.concat([pairs,temp], axis = 1)
del temp
gc.collect()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/4871594 [00:00<?, ?it/s]

...

  0%|          | 0/4871594 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4871594 [00:00<?, ?it/s]

  0%|          | 0/4871594 [00:00<?, ?it/s]

19

In [6]:
pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4871594 entries, 0 to 4871593
Data columns (total 67 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   id_1                        object 
 1   name_1                      object 
 2   latitude_1                  float64
 3   longitude_1                 float64
 4   address_1                   object 
 5   city_1                      object 
 6   state_1                     object 
 7   zip_1                       object 
 8   country_1                   object 
 9   url_1                       object 
 10  phone_1                     object 
 11  categories_1                object 
 12  id_2                        object 
 13  name_2                      object 
 14  latitude_2                  float64
 15  longitude_2                 float64
 16  address_2                   object 
 17  city_2                      object 
 18  state_2                     object 
 19  zip_2                

In [7]:
pairs.to_pickle('./train_pairs.pkl')
#pairs.to_pickle('./train_pairs_sample.pkl')
#pairs.to_pickle('./train_pairs_test.pkl')

In [8]:
cols = ['address_1_missing',
 'address_2_missing',
 'categories_1_count',
 'categories_1_count_diff',
 'categories_2_count',
 'city_1_count',
 'city_1_count_diff',
 'city_1_missing',
 'city_2_count',
 'city_2_missing',
 'country_1_count',
 'country_1_count_diff',
 'distance',
 'latitude_1',
 'latitude_1_count',
 'latitude_2_count',
 'latitude_2_count_diff',
 'longitude_1',
 'longitude_1_count',
 'longitude_2_count',
 'longitude_2_count_diff',
 'phone_1_missing',
 'phone_2_missing',
 'state_1_count',
 'state_1_count_diff',
 'state_2_count',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing',
 'name_1_fuzzy',
 'name_1_fuzzy_partial',
 'address_1_fuzzy',
 'categories_1_fuzzy',
 'categories_1_fuzzy_partial',
 'url_1_fuzzy']
ids = ['id_1','id_2']

<a href="./train_pairs.pkl"> train_pairs pickle </a>